In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import math

In [3]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int,
              'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [4]:
sales = pd.read_csv(r"D:\regression\Project 6\data\kc_house_data.csv", dtype=dtype_dict)

In [5]:
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


# creating new features

In [6]:
# taking square root of sqft_living will decrease the separation between big house and small house.
# The owner may not be exactly twice as happy for getting a house that is twice as big.
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(math.sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(math.sqrt)
#Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) 
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

In [7]:
sales.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'sqft_living_sqrt',
       'sqft_lot_sqrt', 'bedrooms_square', 'floors_square'],
      dtype='object')

In [8]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

# L1 penalty of 5e2

In [9]:
model_all=Lasso(alpha=5e2,normalize=True)

In [10]:
model_all.fit(sales[all_features],sales['price'])

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [11]:
model_all.coef_

array([    0.        ,     0.        ,     0.        ,   134.43931396,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        , 24750.00458561,     0.        ,
       61749.10309071,     0.        ,     0.        ,    -0.        ,
           0.        ])

In [12]:
selected_features=[all_features[i] for i in range(len(model_all.coef_)) if model_all.coef_[i]!=0 ]

In [13]:
print(selected_features)

['sqft_living', 'view', 'grade']


In [14]:
testing = pd.read_csv(r'D:\regression\Project 6\data\kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv(r'D:\regression\Project 6\data\kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv(r'D:\regression\Project 6\data\kc_house_valid_data.csv', dtype=dtype_dict)

###### create the 4 features as we did in line 6

In [15]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(math.sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(math.sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(math.sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(math.sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(math.sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(math.sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [16]:
l1_penalties=np.logspace(1, 7, num=13)

In [17]:
rss_list=[]
for l1_penalty in l1_penalties:
    model = Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features],training['price'])
    prediction=model.predict(validation[all_features])
    error=validation['price']-prediction
    rss=np.sum(error**2)
    rss_list.append((rss,l1_penalty))

In [18]:
min(rss_list) # the value of l1_penalty produced the lowest RSS on VALIDATION data

(398213327300134.94, 10.0)

In [19]:
model=Lasso(alpha=10.0,normalize=True)
model.fit(training[all_features],training['price'])

Lasso(alpha=10.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [20]:
test_prediction=model.predict(testing[all_features])

In [21]:
test_error=testing['price']-test_prediction

In [22]:
test_rss=np.sum(error**2)

In [23]:
print("testing error is",test_rss)

testing error is 1222506859427163.0


In [24]:
print("Number of nonzero coefficients are",np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_))

Number of nonzero coefficients are 15


# Let's try to select model with 7 fetures only

In [25]:
max_nonzeros = 7

In [26]:
l1_penalties=np.logspace(1, 4, num=20)

In [27]:
feature_count=[]
for l1_penalty in l1_penalties:
    model = Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features],training['price'])
    #prediction=model.predict(validation[all_features])
    #error=validation['price']-prediction
    #rss=np.sum(error**2)
    num_features=np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    feature_count.append((num_features,l1_penalty))

In [28]:
feature_count

[(15, 10.0),
 (15, 14.38449888287663),
 (15, 20.6913808111479),
 (15, 29.76351441631318),
 (13, 42.81332398719393),
 (12, 61.58482110660264),
 (11, 88.58667904100822),
 (10, 127.42749857031335),
 (7, 183.29807108324357),
 (6, 263.6650898730358),
 (6, 379.26901907322497),
 (6, 545.5594781168514),
 (5, 784.7599703514607),
 (3, 1128.8378916846884),
 (3, 1623.776739188721),
 (2, 2335.7214690901214),
 (1, 3359.818286283781),
 (1, 4832.930238571752),
 (1, 6951.927961775606),
 (1, 10000.0)]

In [29]:
l1_penalty_min = max([i for i in feature_count if i[0]>max_nonzeros],key=lambda x:x[1])
# penalty smaller than this value will definitely have too many non-zero weights

In [30]:
l1_penalty_max = min([i for i in feature_count if i[0]<max_nonzeros],key=lambda x:x[1])
# penalty larger than this value, we will definitely have too few non-zero weights

In [31]:
print(l1_penalty_min)
print(l1_penalty_max)

(10, 127.42749857031335)
(6, 263.6650898730358)


Exploring narrower range of l1_penalty

In [32]:
rss_list=[]
l1_penalties=np.linspace(l1_penalty_min[1],l1_penalty_max[1],20)
for l1_penalty in l1_penalties:
    model = Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features],training['price'])    
    num_features=np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    if num_features == max_nonzeros:
        prediction=model.predict(validation[all_features])
        error=validation['price']-prediction
        rss=np.sum(error**2)
        rss_list.append((rss,l1_penalty))

In [33]:
rss_list

[(440037365263316.56, 156.10909673930755),
 (440777489641605.25, 163.2794962815561),
 (441566698090139.9, 170.44989582380464),
 (442406413188666.25, 177.6202953660532),
 (443296716874315.06, 184.79069490830176),
 (444239780526141.6, 191.96109445055032),
 (445230739842614.25, 199.13149399279888)]

In [34]:
min(rss_list)

(440037365263316.56, 156.10909673930755)

In [35]:
model = Lasso(alpha=156.10909673930755, normalize=True)
model.fit(training[all_features],training['price'])

Lasso(alpha=156.10909673930755, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=True, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [36]:
selected_features=[all_features[i] for i in range(len(model.coef_)) if model.coef_[i]!=0 ]

In [37]:
print(selected_features)

['bathrooms', 'sqft_living', 'waterfront', 'view', 'grade', 'yr_built']


# Implementing LASSO from scratch using coordinate descent

In [38]:
def get_numpy_data(df,features,output):
    '''
    i/p: df: data table in data frame format
            features: x input name list. According to this list a feature matrix will be created in numpy 2D aray
            output: single target name which will be converted into numpy vector
    o/p: feature matrix, y vector in numpy array format'''
    df['constant']=1   #add a constant field for the intercept
    features=['constant']+features
    feature_matrix=df.as_matrix(columns=features)
    y_vector=df[output].values
    return feature_matrix,y_vector

In [39]:
def predict_output(feature_matrix,weights):
    '''
    i/p: feature_matrix: matrix F of equation Fw=y
         weights is a one D w vector in above equation
    o/p: predicted value y in above eq.
    '''
    y_hat=np.dot(feature_matrix,weights)
    return y_hat

In [40]:
def normalize_features(features):
    norms = np.linalg.norm(features,axis=0)
    normlized_features = features / norms
    return (normlized_features, norms)

In [41]:
#test
test_f=np.array([[3,6,9],[4,8,12]])
print("array")
print(test_f)
#result
print("formula")
norms=[math.sqrt(9+16),math.sqrt(36+64),math.sqrt(81+144)]
print(norms)
for i in range(2):
    for j in range(3):
        print(test_f[i,j]/norms[j],end=" ")
    print()
# result using function
print("function")
print(normalize_features(test_f))

array
[[ 3  6  9]
 [ 4  8 12]]
formula
[5.0, 10.0, 15.0]
0.6 0.6 0.6 
0.8 0.8 0.8 
function
(array([[0.6, 0.6, 0.6],
       [0.8, 0.8, 0.8]]), array([ 5., 10., 15.]))


In [42]:
math.sqrt(9+36+81)

11.224972160321824

$  
\hat{w_j} = 
     \begin{cases}
       \rho_j + \frac{\lambda}{2}; &\quad\rho_j<-\frac{\lambda}{2}\\
       0; &\quad-\frac{\lambda}{2}\le\rho_j\le\frac{\lambda}{2}\\
       \rho_j - \frac{\lambda}{2}; &\quad\rho_j>+\frac{\lambda}{2}
     \end{cases}
$

In [43]:
features = ['sqft_living','bedrooms']
output = 'price'

In [44]:
features_matrix, output_array = get_numpy_data(sales, features, output)

In [45]:
normalized_features_matrix,norms = normalize_features(features_matrix)

In [46]:
print(normalized_features_matrix)

[[0.00680209 0.00353021 0.00583571]
 [0.00680209 0.00768869 0.00583571]
 [0.00680209 0.00230361 0.00389048]
 ...
 [0.00680209 0.00305154 0.00389048]
 [0.00680209 0.00478673 0.00583571]
 [0.00680209 0.00305154 0.00389048]]


In [47]:
print(features_matrix)

[[1.00e+00 1.18e+03 3.00e+00]
 [1.00e+00 2.57e+03 3.00e+00]
 [1.00e+00 7.70e+02 2.00e+00]
 ...
 [1.00e+00 1.02e+03 2.00e+00]
 [1.00e+00 1.60e+03 3.00e+00]
 [1.00e+00 1.02e+03 2.00e+00]]


In [48]:
print(norms)

[1.47013605e+02 3.34257264e+05 5.14075870e+02]


In [49]:
weights = [1,4,1]

In [50]:
prediction = predict_output(normalized_features_matrix, weights)

In [51]:
def calculate_ro(j,feature_matrix, output, prediction, weights):
    ro=np.dot(feature_matrix[:,j],(output-prediction+(weights[j]*feature_matrix[:,j])))
    return ro

In [52]:
ro_1=calculate_ro(1,normalized_features_matrix,output_array,prediction,weights)
ro_2=calculate_ro(2,normalized_features_matrix,output_array,prediction,weights)
print(ro_1,ro_2)

87939470.82325175 80966698.66623946


In [53]:
ro_1>ro_2

True

In [54]:
print("Range of L1 penalty which would set w2=0 but not w1 is",[2*ro_2,2*ro_1])

Range of L1 penalty which would set w2=0 but not w1 is [161933397.3324789, 175878941.6465035]


In [55]:
print("Range of L1 penalty which would set both weights equal to zero is greater than or equal to",2*ro_1)

Range of L1 penalty which would set both weights equal to zero is greater than or equal to 175878941.6465035


# Single Coordinate Descent Step

In [56]:
def lasso_coordinate_descent_step(j, feature_matrix, output, weights, l1_penalty):
    prediction = predict_output(feature_matrix, weights)
    ro_j = calculate_ro(j, feature_matrix, output,prediction, weights)
    
    if j == 0: # intercept -- do not regularize
        new_weight_j = ro_j
    elif ro_j < -l1_penalty/2.:
        new_weight_j = ro_j + (l1_penalty/2.)
    elif ro_j > l1_penalty/2.:
        new_weight_j = ro_j - (l1_penalty/2.)
    else:
        new_weight_j = 0.
    
    return new_weight_j

In [57]:
#test
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


# Cyclical coordinate descent

In [58]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    max_change = tolerance+1
    weights=np.array(initial_weights)
    while max_change>tolerance:
        max_change=0
        for j in range(len(weights)):
            new_w=lasso_coordinate_descent_step(j,feature_matrix, output, weights, l1_penalty)
            w_change=np.abs(new_w-weights[j])
            if w_change>max_change: max_change=w_change
            weights[j]=new_w
    return weights

In [59]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [60]:
simple_feature_matrix, output=get_numpy_data(sales,simple_features,my_output)

In [61]:
normalized_features_matrix,norms=normalize_features(simple_feature_matrix)

In [62]:
weights=lasso_cyclical_coordinate_descent(normalized_features_matrix,output,initial_weights,l1_penalty,tolerance)

In [63]:
print("training rss is",np.sum((output-predict_output(normalized_features_matrix,weights))**2))

training rss is 1630492476715386.5


In [64]:
print("weights are",weights)

weights are [21624997.95951909 63157247.20788956        0.        ]


In [65]:
print("weight for #bedrooms is zero")

weight for #bedrooms is zero


# Evaluating LASSO fit with more features


In [66]:
features = ['bedrooms',
            'bathrooms',
            'sqft_living',
            'sqft_lot',
            'floors',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

In [67]:
feature_matrix_train, output_train=get_numpy_data(training,features,my_output)

In [68]:
normalize_features_train,norms=normalize_features(feature_matrix_train)

In [69]:
l1_penalty = 1e7
initialize_weights = np.zeros(14)
tolerance = 1

In [70]:
weights1e7=lasso_cyclical_coordinate_descent(normalize_features_train,output_train,initialize_weights,l1_penalty,tolerance)

In [71]:
weights1e7

array([23864692.50953839,        0.        ,        0.        ,
       30495548.1325472 ,        0.        ,        0.        ,
        1901633.61475594,  5705765.01673266,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [72]:
pd.Series(weights1e7,index=['intercept']+ features) 

intercept        2.386469e+07
bedrooms         0.000000e+00
bathrooms        0.000000e+00
sqft_living      3.049555e+07
sqft_lot         0.000000e+00
floors           0.000000e+00
waterfront       1.901634e+06
view             5.705765e+06
condition        0.000000e+00
grade            0.000000e+00
sqft_above       0.000000e+00
sqft_basement    0.000000e+00
yr_built         0.000000e+00
yr_renovated     0.000000e+00
dtype: float64

In [73]:
selected_features=[features[i-1] for i in range(1,len(weights1e7)) if weights1e7[i]!=0 ]

In [74]:
print(selected_features)

['sqft_living', 'waterfront', 'view']


In [75]:
weights1e7[4]!=0 

False

In [76]:
l1_penalty=1e8

In [77]:
weights1e8=lasso_cyclical_coordinate_descent(normalize_features_train,output_train,initialize_weights,l1_penalty,tolerance)

In [78]:
weights1e8

array([53621004.68971471,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [79]:
selected_features=[features[i-1] for i in range(1,len(weights1e8)) if weights1e8[i]!=0 ]

In [80]:
print(selected_features)

[]


In [81]:
l1_penalty = 1e4
tolerance = 5e5

In [82]:
weights1e4=lasso_cyclical_coordinate_descent(normalize_features_train,output_train,initialize_weights,l1_penalty,tolerance)

In [83]:
weights1e4

array([ 57481091.13302065, -13652628.5402242 ,  12462713.07126241,
        57942788.37331225,  -1475769.69427563,  -4904547.7554655 ,
         5349050.18636169,   5845253.56213634,   -416038.96981262,
         2682274.59488504,    242649.68555067,  -1285549.66768123,
       -54779474.22768356,   2167703.06610233])

In [84]:
selected_features=[features[i-1] for i in range(1,len(weights1e4)) if weights1e4[i]!=0 ]

In [85]:
print(selected_features)

['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


In [86]:
selected_features==features

True

# Rescaling learned weights

In [87]:
norms

array([9.87977733e+01, 3.46770818e+02, 2.22709592e+02, 2.25597973e+05,
       4.34516277e+06, 1.55954320e+02, 9.05538514e+00, 8.16026960e+01,
       3.43512736e+02, 7.65904694e+02, 1.95467912e+05, 5.24647562e+04,
       1.94732030e+05, 4.09449564e+04])

In [88]:
weights1e7_normalized = weights1e7 / norms
weights1e8_normalized = weights1e8 / norms
weights1e4_normalized = weights1e4 / norms

In [89]:
weights1e7_normalized[3]

135.1765164613825

In [90]:
(test_feature_matrix, test_output) = get_numpy_data(testing, features, 'price')

In [91]:
print ("test rss of 1e7 model is %e"%sum((test_output - predict_output(test_feature_matrix,weights1e7_normalized) )**2))
print ("test rss of 1e4 model is %e"%sum((test_output - predict_output(test_feature_matrix,weights1e4_normalized) )**2))
print ("test rss of 1e8 model is %e"%sum((test_output - predict_output(test_feature_matrix,weights1e8_normalized) )**2))

test rss of 1e7 model is 1.631036e+14
test rss of 1e4 model is 1.290853e+14
test rss of 1e8 model is 2.847189e+14
